# 0. Model Description
### Build a simple feed-forward deep NN that could predict architects' names based on their designs. The model could be enhanced by using CNNs or architectures like LeNet, AlexNet, VGG, ResNet... and of course by building a great dataset of different architects' designs.
### Dataset: https://www.kaggle.com/datasets/galaxyarchitects/architects-dataset

# 1. Import Libraries

In [ ]:
import os
import scipy
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense

### 1.1 Switch to TensorFlow GPU

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Limit GPU memory growth
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

### 2. Loading Dataset

In [ ]:
dataset = "/kaggle/input/architects-dataset"

In [ ]:
def listdirs(dataset):
    subfolders = []
    for file in os.listdir(dataset):
        d = os.path.join(dataset, file)
        if os.path.isdir(d):
            subfolders.append(d) #(d.replace("\\", "/"))
            subfolders.extend(listdirs(d))
    return subfolders

subfolders_list = listdirs(dataset)
#remove = str.maketrans('', '', "[]'")
#subfolders_list = subfolders_list.translate(remove)
print(subfolders_list)

# 3. Generating Dataset

In [ ]:
img_height = 180
img_width = 180
batch_size = 32

#tf.data.Dataset??
data = tf.keras.utils.image_dataset_from_directory(dataset, batch_size=batch_size, image_size = (img_height, img_width))

In [ ]:
data_iterator = data.as_numpy_iterator()

In [ ]:
#Get another batch from the iterator
batch = data_iterator.next()

#Images represented as numpy arrays
batch[0].shape

In [ ]:
# plot data from a batch
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

# 4. Preprocessing Data

### 4.1. Scaling Data

In [ ]:
data = data.map(lambda x,y: (x/255.0, y))
data.as_numpy_iterator().next()[0].max()

### 4.2. One-Hot-Encoding

In [ ]:
all_labels = []
for images, labels in data:
    all_labels.append(labels)

all_labels = tf.concat(all_labels, axis=0)

num_classes = tf.reduce_max(all_labels) + 1

def one_hot_encode(labels, num_classes):
    return tf.one_hot(labels, num_classes)

one_hot_encoded_labels = one_hot_encode(all_labels, num_classes)

label_dataset = tf.data.Dataset.from_tensor_slices(one_hot_encoded_labels)

full_dataset = tf.data.Dataset.zip((data, label_dataset))

In [ ]:
# Apply the split function to the zipped dataset
image_dataset = full_dataset.map(lambda x, y: x)
label_dataset = full_dataset.map(lambda x, y: y)

### 4.3. Splitting Data

In [ ]:
len(image_dataset) #number of batches

train_size = int(len(image_dataset)*0.7)
val_size = int(len(image_dataset)*0.2)
test_size = int(len(image_dataset)*0.1)

train_size+val_size+test_size

train = image_dataset.take(train_size)
val = image_dataset.skip(train_size).take(val_size)
test = image_dataset.skip(train_size+val_size).take(test_size)

len(test)

# 5. Deep Learning Model (FeedForward Neural Network)

In [ ]:
model = Sequential()

model.add(Dense(10, input_shape=(img_height,img_width,3), activation = 'relu'))

model.add(Dense(10, activation = 'relu'))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(10, activation = 'relu'))

model.add(Flatten())

model.add(Dense(25, activation = 'softmax'))

In [ ]:
model.compile('adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

# 6. Train the Model

In [ ]:
logdir = 'logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
hist = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback])

### 6.1. Plot the Model's Performance

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

# 7. Testing the Model

In [ ]:
classes_list = ["Alvaro Siza", "Antonio Gaudi", "Bernard Tschumi", "Bjarke Ingels", "Daniel Libeskind", 
               "Eero Saarinen", "Frank Gehry", "Gordon Bunshaft", "I M Pei", "Jean Nouvel", 
               "Kenzo Tange", "Le Corbusier", "Michael Graves", "Mies van der Rohe", "Norman Foster", 
               "Oscar Niemeyer", "Peter Eisenman", "Philip Johnson", "Rem Koolhaas", "Renzo Piano", 
               "Richard Meier", "Santiago Calatrava", "Tadao Ando", "Toyo Ito", "Zaha Hadid"]

img = keras.utils.load_img(
    "/kaggle/input/architects-dataset/Alvaro Siza/1 (5).jpg", target_size=(img_height, img_width)
)

img_array = keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create batch axis
img_array /= 255.0 

predictions = model.predict(img_array)
#predicted_class_index = tf.argmax(predictions, axis=-1)

for class_name, prob in zip(classes_list, predictions[0]):
    print(f"{class_name}: {prob}")

predicted_class_index = tf.math.argmax(predictions[0], axis=-1)

predicted_class = classes_list[predicted_class_index]

confidence_percentage = 100 * predictions[0][predicted_class_index]

print(f"This image is classified as {predicted_class} with confidence {confidence_percentage:.2f}%.")

# 8. Saving the Model

In [ ]:
from tensorflow.keras.models import load_model
model.save(os.path.join('models', 'architects.h5'))

# 9. Loading the Model

In [ ]:
new_model = load_model(os.path.join('models', 'architects.h5'))